<a href="https://colab.research.google.com/github/asifsiraz/ML-projects/blob/main/Sparse_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.regularizers import l1  # Import the L1 regularization

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define the model with batch normalization and ReLU activation, and L1 regularization
def create_model_with_l1():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation='relu', kernel_initializer='he_normal', activity_regularizer =l1(0.001)),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu', kernel_initializer='he_normal', activity_regularizer=l1(0.001)),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu', kernel_initializer='he_normal', activity_regularizer=l1(0.001)),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax',activity_regularizer=l1(0.001))
    ])
    return model

learning_rate = 0.001

# Train the model with L1 regularization
model_with_l1 = create_model_with_l1()
model_with_l1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_with_l1.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)

# Function to calculate the percentage of active nodes in each layer using forward pass
def calculate_active_nodes(model, x):
    active_nodes_counts = []
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Perform forward pass
            x = layer(x)
            # Identify active nodes (non-zero activations) for the current layer
            active_nodes = np.sum(np.any(x.numpy() != 0,axis=0))
            active_nodes_counts.append(active_nodes)
    return active_nodes_counts

# Create a sample batch for the forward pass
sample_batch_flattened = train_images[:64].reshape(64, -1)

# Calculate the number of active nodes for each layer using forward pass
active_nodes_counts = calculate_active_nodes(model_with_l1, sample_batch_flattened)
for i, count in enumerate(active_nodes_counts, start=1):
    print(f"Layer {i}: {count} active nodes")

def calculate_active_nodes_percentages(model, x):
    active_nodes_Counts=[]
    active_nodes_percentages = []
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Perform forward pass
            x = layer(x)
            # Identify active nodes (non-zero activations) for the current layer
            active_nodes = np.sum(np.any(x.numpy() != 0, axis=0))
            total_nodes = x.shape[-1]
            active_nodes_percentage = (active_nodes / total_nodes) * 100
            active_nodes_percentages.append(active_nodes_percentage)

    return active_nodes_percentages


active_nodes_percentages = calculate_active_nodes_percentages(model_with_l1, sample_batch_flattened)
for i, percentage in enumerate(active_nodes_percentages, start=1):
    print(f"Layer {i}: {percentage:.2f}% active nodes")


def calculate_sparsity(model):
    total_params = 0
    zero_params = 0
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            # Get the layer's weights
            weights = layer.get_weights()[0]
            total_params += np.prod(weights.shape)
            zero_params += np.sum(np.abs(weights) < 1e-5)  # Count the number of near-zero weights
    sparsity = zero_params / total_params * 100
    return sparsity

# Calculate the sparsity of the model
sparsity = calculate_sparsity(model_with_l1)
print(f"Model Sparsity: {sparsity:.2f}%")


Epoch 1/10
750/750 [==============================] - 7s 7ms/step - loss: 0.3968 - accuracy: 0.9242 - val_loss: 0.2595 - val_accuracy: 0.9554
Epoch 2/10
750/750 [==============================] - 5s 7ms/step - loss: 0.2277 - accuracy: 0.9609 - val_loss: 0.2311 - val_accuracy: 0.9637
Epoch 3/10
750/750 [==============================] - 5s 7ms/step - loss: 0.1885 - accuracy: 0.9671 - val_loss: 0.2020 - val_accuracy: 0.9623
Epoch 4/10
750/750 [==============================] - 5s 6ms/step - loss: 0.1637 - accuracy: 0.9706 - val_loss: 0.1904 - val_accuracy: 0.9675
Epoch 5/10
750/750 [==============================] - 6s 8ms/step - loss: 0.1474 - accuracy: 0.9746 - val_loss: 0.1861 - val_accuracy: 0.9657
Epoch 6/10
750/750 [==============================] - 5s 6ms/step - loss: 0.1374 - accuracy: 0.9759 - val_loss: 0.1734 - val_accuracy: 0.9677
Epoch 7/10
750/750 [==============================] - 6s 8ms/step - loss: 0.1276 - accuracy: 0.9779 - val_loss: 0.1798 - val_accuracy: 0.9630
Epoch 